In [70]:
import shapefile as shp
import json
import math

In [71]:
s = shp.Reader("shapefiles/Builtup_Kalonda.shp")

In [72]:
fields = s.fields[1:]
f = fields[0]
fields, [f[0] for f in fields]

([['OBJECTID', 'N', 10, 0],
  ['GlobalID', 'C', 38, 0],
  ['Uniq_Id', 'C', 50, 0],
  ['State_Code', 'C', 2, 0],
  ['District_C', 'C', 3, 0],
  ['Tehsil_Cod', 'C', 5, 0],
  ['Block_Code', 'C', 5, 0],
  ['Village_Co', 'C', 6, 0],
  ['LGD_Code', 'C', 6, 0],
  ['Built_Up_A', 'N', 5, 0],
  ['Village_Na', 'C', 50, 0],
  ['Area_Desc', 'C', 50, 0],
  ['Name', 'C', 50, 0],
  ['Owner_Name', 'C', 50, 0],
  ['Property_I', 'C', 50, 0],
  ['Property_C', 'C', 50, 0],
  ['Area_Sqm', 'F', 19, 11],
  ['Roof_type', 'N', 5, 0],
  ['No_Floors', 'N', 5, 0],
  ['Remarks', 'C', 50, 0],
  ['Adl_Info', 'C', 50, 0],
  ['SHAPE_Leng', 'F', 19, 11],
  ['SHAPE_Area', 'F', 19, 11],
  ['GP_Code', 'C', 6, 0],
  ['GP_Name', 'C', 50, 0]],
 ['OBJECTID',
  'GlobalID',
  'Uniq_Id',
  'State_Code',
  'District_C',
  'Tehsil_Cod',
  'Block_Code',
  'Village_Co',
  'LGD_Code',
  'Built_Up_A',
  'Village_Na',
  'Area_Desc',
  'Name',
  'Owner_Name',
  'Property_I',
  'Property_C',
  'Area_Sqm',
  'Roof_type',
  'No_Floors',
  '

In [73]:
shapes = s.shapes()
sh = shapes[0]

In [74]:
sh.bbox, sh.parts, sh.points, sh.shapeType

([8646538.384300001, 3317213.4416000023, 8646543.103, 3317220.3533000024],
 [0],
 [(8646538.384300001, 3317219.283399999),
  (8646539.8134, 3317220.3532999977),
  (8646543.103, 3317214.365699999),
  (8646541.131499998, 3317213.4416000023),
  (8646539.5948, 3317216.9070999995),
  (8646538.384300001, 3317219.283399999)],
 5)

In [75]:
records = s.records()
r=records[0]
r.GP_Code

'63317'

In [76]:
data = []

# add buildings from shapefile to data
for i in range(len(records)):
    one = {}
    one["points"] = [{"x": p[0], "y": p[1]} for p in shapes[i].points]
    one["bbox"] = {
        "x1": shapes[i].bbox[0],
        "y1": shapes[i].bbox[1],
        "x2": shapes[i].bbox[2],
        "y2": shapes[i].bbox[3],
    }
    one["type"] = "building"
    for j in range(len(fields)):
        one[fields[j][0]] = records[i][j]
    data.append(one)


In [77]:
# add openstreetmap roads to data
from OSMPythonTools.nominatim import Nominatim
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass

nominatim = Nominatim()
res = nominatim.query('Kalonda, UP')

In [78]:
bbox = res.toJSON()[0]["boundingbox"]
print(bbox)
bbox[1], bbox[2] = bbox[2], bbox[1]

['28.5254074', '28.5654074', '77.6533559', '77.6933559']


In [79]:
roads = overpassQueryBuilder(bbox=bbox, elementType='way', selector='"highway"~"motorway|trunk|primary|secondary|tertiary|unclassified|residential|service|living_street"', out='body')
print(roads)

overpass = Overpass()
result = overpass.query(roads)
print(result)

(way["highway"~"motorway|trunk|primary|secondary|tertiary|unclassified|residential|service|living_street"](28.5254074,77.6533559,28.5654074,77.6933559);); out body;


In [80]:
def latlong_to_pseudo_mercator(lat, long):
    x = long * 20037508.34 / 180
    y = math.log(math.tan((90 + lat) * math.pi / 360)) / (math.pi / 180)
    y = y * 20037508.34 / 180
    return x, y

els = result.elements()
coordinates = []
for e in els:
    coordinates.append(list(map(lambda c: latlong_to_pseudo_mercator(*c[::-1]), e.geometry()['coordinates'])))

In [81]:
coordinates

[[(8648411.238525555, 3317481.9328394504),
  (8648414.355471296, 3317482.0595596186),
  (8648444.411733808, 3317480.412197524),
  (8648469.792577704, 3317479.1449960554),
  (8648494.394185167, 3317478.2579550976),
  (8648513.986415543, 3317476.864033712),
  (8648534.469201846, 3317474.583071759),
  (8648555.397266112, 3317471.415069692),
  (8648575.323454961, 3317467.7401882336),
  (8648597.921311589, 3317462.41794792),
  (8648627.86625461, 3317455.955230391),
  (8648633.432229148, 3317455.321630801),
  (8648638.998203687, 3317456.0819503088),
  (8648649.128277348, 3317463.811867797),
  (8648656.252724757, 3317469.387548765),
  (8648663.488491658, 3317473.3158708434),
  (8648677.96002546, 3317480.2854773747),
  (8648690.87308639, 3317485.2275642124),
  (8648698.22017278, 3317486.2413258394),
  (8648707.459690515, 3317487.381807767),
  (8648716.365249777, 3317487.255087545),
  (8648737.627272516, 3317485.35428441),
  (8648791.060628088, 3317478.3846752313),
  (8648818.33390333, 3317474.

In [82]:
for c in coordinates:
    one = {}
    one["points"] = [{"x": p[0], "y": p[1]} for p in c]
    one["bbox"] = {
        "x1": min([p[0] for p in c]),
        "y1": min([p[1] for p in c]),
        "x2": max([p[0] for p in c]),
        "y2": max([p[1] for p in c]),
    }
    one["type"] = "road"
    data.append(one)

In [83]:
json.dump({"structures": data}, open("village_data.json", "w"))
data

[{'points': [{'x': 8646538.384300001, 'y': 3317219.283399999},
   {'x': 8646539.8134, 'y': 3317220.3532999977},
   {'x': 8646543.103, 'y': 3317214.365699999},
   {'x': 8646541.131499998, 'y': 3317213.4416000023},
   {'x': 8646539.5948, 'y': 3317216.9070999995},
   {'x': 8646538.384300001, 'y': 3317219.283399999}],
  'bbox': {'x1': 8646538.384300001,
   'y1': 3317213.4416000023,
   'x2': 8646543.103,
   'y2': 3317220.3533000024},
  'type': 'building',
  'OBJECTID': 52390,
  'GlobalID': '{00000000-0000-0000-0000-000000000000}',
  'Uniq_Id': '',
  'State_Code': '9',
  'District_C': '144',
  'Tehsil_Cod': '742',
  'Block_Code': '1126',
  'Village_Co': '120196',
  'LGD_Code': '120196',
  'Built_Up_A': 1,
  'Village_Na': 'Kalonda',
  'Area_Desc': '',
  'Name': '',
  'Owner_Name': '',
  'Property_I': '',
  'Property_C': '',
  'Area_Sqm': 9.83786452932,
  'Roof_type': 1,
  'No_Floors': 1,
  'Remarks': '',
  'Adl_Info': '',
  'SHAPE_Leng': 17.252085894,
  'SHAPE_Area': 12.7944454348,
  'GP_Code

In [84]:
d = json.load(open("village_data.json"))
d

{'structures': [{'points': [{'x': 8646538.384300001, 'y': 3317219.283399999},
    {'x': 8646539.8134, 'y': 3317220.3532999977},
    {'x': 8646543.103, 'y': 3317214.365699999},
    {'x': 8646541.131499998, 'y': 3317213.4416000023},
    {'x': 8646539.5948, 'y': 3317216.9070999995},
    {'x': 8646538.384300001, 'y': 3317219.283399999}],
   'bbox': {'x1': 8646538.384300001,
    'y1': 3317213.4416000023,
    'x2': 8646543.103,
    'y2': 3317220.3533000024},
   'type': 'building',
   'OBJECTID': 52390,
   'GlobalID': '{00000000-0000-0000-0000-000000000000}',
   'Uniq_Id': '',
   'State_Code': '9',
   'District_C': '144',
   'Tehsil_Cod': '742',
   'Block_Code': '1126',
   'Village_Co': '120196',
   'LGD_Code': '120196',
   'Built_Up_A': 1,
   'Village_Na': 'Kalonda',
   'Area_Desc': '',
   'Name': '',
   'Owner_Name': '',
   'Property_I': '',
   'Property_C': '',
   'Area_Sqm': 9.83786452932,
   'Roof_type': 1,
   'No_Floors': 1,
   'Remarks': '',
   'Adl_Info': '',
   'SHAPE_Leng': 17.25208

In [85]:
minX, minY, maxX, maxY = 1000000000, 1000000000, -1000000000, -1000000000
for i in d["structures"]:
    if i["type"] == "building":
        for p in i["points"]:
            minX = min(minX, p["x"])
            minY = min(minY, p["y"])
            maxX = max(maxX, p["x"])
            maxY = max(maxY, p["y"])
minX, minY, maxX, maxY

(8646009.2088, 3317023.326200001, 8647274.471, 3318574.2629999965)